In [16]:
import typing as tp
import numpy as np
import pandas as pd
import os

from collections import defaultdict
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import re
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Load Data

In [62]:
data = pd.read_csv('/home/iris/paulshab/Mynalabs_NLP/data/data_elonmusk.csv', encoding='windows-1252')
df = data
data.head()

,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk


## Hashtag Extraction using Regex

In [63]:
df.sample(4)

,row ID,Tweet,Time,Retweet from,User
538,Row538,RT @SpaceX: Falcon 9 and Dragon on Pad 39A at ...,2017-06-01 19:28:45,SpaceX,elonmusk
3020,Row3020,Always admired Margaret Thatcher -- she was to...,2013-04-09 03:43:28,NaN,elonmusk
2535,Row2535,RT @mashable: Scary Smart Video Predicts Autom...,2014-08-19 18:17:02,mashable,elonmusk
309,Row309,Stop biting my finger dammit \nhttps://t.co/Xh...,2017-07-08 13:24:06,NaN,elonmusk


In [67]:
df['hashtag'] = df['Tweet'].apply(lambda x: re.findall(r"#(\w+)", x))

## Tweet Processing Libaray

In [ ]:
# #installing tweet-preprocessor
!pip install tweet-preprocessor

In [42]:
from ekphrasis.classes.segmenter import Segmenter
import preprocessor as p
#segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [64]:
#forming a separate feature for cleaned tweets
for i,v in enumerate(df['Tweet']):  # i --> num of tweet, v = tweet
    df.loc[i,'Cleaned_Tweet'] = p.clean(v)

## Tokenization , Removal of Digits, Stop Words and Punctuations


In [110]:
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer


[nltk_data] Downloading package wordnet to /home/iris/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /home/iris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [116]:
# Remove Digits and lower the text
df['Cleaned_Tweet'] = df['Cleaned_Tweet'].astype(str).str.replace('\d+', '')
df['Cleaned_Tweet'] = df['Cleaned_Tweet'].str.lower()

/home/iris/paulshab/NLP_env/nlp_environ/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [124]:
df.sample(4)

,row ID,Tweet,Time,Retweet from,User,Cleaned_Tweet,hashtag
1613,Row1613,RT @SpaceX: Targeting launch tomorrow of @Eute...,2016-06-15 10:37:53,SpaceX,elonmusk,: targeting launch tomorrow of & abs communica...,[]
439,Row439,"@ID_AA_Carmack @mmeijeri The faster, the bette...",2017-06-10 01:27:37,NaN,elonmusk,"the faster, the better. gravity losses.",[]
1228,Row1228,"Unveil happening at sunset, because that's whe...",2016-10-28 02:37:44,NaN,elonmusk,"unveil happening at sunset, because that's whe...",[]
2458,Row2458,Nothing against all the oatmeal-raisin cookies...,2014-12-21 01:55:27,NaN,elonmusk,nothing against all the oatmeal-raisin cookies...,[]


In [127]:
df.iloc[[3211]]['Cleaned_Tweet']

3211    btw, i don't think apple is doomed. just won't...
Name: Cleaned_Tweet, dtype: object

In [135]:
data.iloc[3214,5]

'these articles in space news describe why ariane vs : and'

In [136]:
data.iloc[3214,1]

'These articles in Space News describe why Ariane 6 vs 5: http://t.co/IaeYXXK9 and http://t.co/ghs3FG8w'

In [137]:
len(data)

3218

### Operations
- deleted hashtags ( not always good)
- used tweet-preprocessor removes
    - URLs   
    - Hashtags
    - Mentions
    - Reserved words (RT, FAV)
    - Emojis
    - Smileys
- removed numbers
- converted to lower

### Questions
- should you fill in Abbreviations (btw, imho ..)
- Should you use Lemmatization?
- should you use Stop Words?
